In [68]:
import os
import sys
import clr
import pandas as pd

clr.AddReference("System")  # Make sure to add a reference to the System namespace
from System import String, Array


In [2]:

ETABS_dll_path = R"C:\Program Files\Computers and Structures\ETABS 20\ETABSv1.dll"

from enum import Enum

clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

# set the following path to the installed ETABS program directory
clr.AddReference(ETABS_dll_path)
from ETABSv1 import *

In [3]:
# key variables
ETABS_model_folder_path = R"test_models\ETABS_gravity"
ETABS_model_file_path = R"test_models\ETABS_gravity\2023-08-29_Gravity Model.EDB"
ProgramPath = R"C:\Program Files\Computers and Structures\ETABS 20\ETABS.exe"

In [4]:
# Boiler plate code to open ETABs File 

# create API helper object
helper = cHelper(Helper())
# create instance of ETABs object from specified path
myETABSObject = cOAPI(helper.CreateObject(ProgramPath))

# start ETABS application
myETABSObject.ApplicationStart()

# create SapModel object
SapModel = cSapModel(myETABSObject.SapModel)

# initialize model
SapModel.InitializeNewModel()

# open EtAbs model
File = cFile(SapModel.File)
ret = File.OpenFile(ETABS_model_file_path)


In [5]:
# run model (this will create the analysis model)
Analyze = cAnalyze(SapModel.Analyze)  ###
ret = Analyze.RunAnalysis()

In [6]:
Results = cAnalysisResults(SapModel.Results)
Setup = cAnalysisResultsSetup(Results.Setup)

In [7]:
# count Linear Static Load Cases
LinearStatic=1
LoadCases = cLoadCases(SapModel.LoadCases)
ret = LoadCases.Count(eLoadCaseType(LinearStatic))
print(f"Load Case qty: {ret}")

Load Case qty: 9


In [21]:
# get linear static load case names
LC_names = []
num_names = 0
[ret, num_names, LC_names] = LoadCases.GetNameList(num_names, LC_names, eLoadCaseType(LinearStatic))

print(ret)
print("\n")
print(num_names)
print("\n")
print(type(LC_names))
print(LC_names[1])
for name in LC_names:
    print(name)

ETABS_LC = [name  for name in LC_names]

print(ETABS_LC)


0


9


<class 'System.String[]'>
Mech
Const
Mech
Dead
Live
Roof Live
S Dead
EQ-X_R=5
EQ-Y_R=5
~LLRF
['Const', 'Mech', 'Dead', 'Live', 'Roof Live', 'S Dead', 'EQ-X_R=5', 'EQ-Y_R=5', '~LLRF']


In [103]:
FrameObj = cFrameObj(SapModel.FrameObj)

NumberNames = 0
MyNames = []
PropName = []
StoryName = []
PointName1 = []
PointName2 = []
Point1X = []
Point1Y = []
Point1Z = []
Point2X = []
Point2Y = []
Point2Z = []
Angle = []
Offset1X = []
Offset2X = []
Offset1Y = []
Offset2Y = []
Offset1Z = []
Offset2Z = []
CardinalPoint = []

[
    ret,
    NumberNames,
    MyNames,
    PropName,
    StoryName,
    PointName1,
    PointName2,
    Point1X,
    Point1Y,
    Point1Z,
    Point2X,
    Point2Y,
    Point2Z,
    Angle,
    Offset1X,
    Offset2X,
    Offset1Y,
    Offset2Y,
    Offset1Z,
    Offset2Z,
    CardinalPoint
] = FrameObj.GetAllFrames(
    NumberNames,
    MyNames,
    PropName,
    StoryName,
    PointName1,
    PointName2,
    Point1X,
    Point1Y,
    Point1Z,
    Point2X,
    Point2Y,
    Point2Z,
    Angle,
    Offset1X,
    Offset2X,
    Offset1Y,
    Offset2Y,
    Offset1Z,
    Offset2Z,
    CardinalPoint
)

In [26]:
print(ret)

0


In [37]:
# explore frame data
def convert_system_array_to_list(sys_str):
    p_str = [item for item in sys_str]
    return p_str


#create pandas data frame
data = {
    "MyNames": convert_system_array_to_list(MyNames),
    "PropName": convert_system_array_to_list(PropName),
    "StoryName": convert_system_array_to_list(StoryName),
    "PointName1": convert_system_array_to_list(PointName1),
    "PointName2": convert_system_array_to_list(PointName2),
    "Point1X": convert_system_array_to_list(Point1X),
    "Point1Y": convert_system_array_to_list(Point1Y),
    "Point1Z": convert_system_array_to_list(Point1Z),
    "Point2X": convert_system_array_to_list(Point2X),
    "Point2Y": convert_system_array_to_list(Point2Y),
    "Point2Z": convert_system_array_to_list(Point2Z),
    "Angle": convert_system_array_to_list(Angle),
    "Offset1X": convert_system_array_to_list(Offset1X),
    "Offset2X": convert_system_array_to_list(Offset2X),
    "Offset1Y": convert_system_array_to_list(Offset1Y),
    "Offset2Y": convert_system_array_to_list(Offset2Y),
    "Offset1Z": convert_system_array_to_list(Offset1Z),
    "Offset2Z": convert_system_array_to_list(Offset2Z),
    "CardinalPoint": convert_system_array_to_list(CardinalPoint)
}

frames_df = pd.DataFrame(data)
frames_df.head()

,MyNames,PropName,StoryName,PointName1,PointName2,Point1X,Point1Y,Point1Z,Point2X,Point2Y,Point2Z,Angle,Offset1X,Offset2X,Offset1Y,Offset2Y,Offset1Z,Offset2Z,CardinalPoint
0,47,24Col8ksi,Level_2,587,58,164.04,3300.0,0.0,164.04,3300.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1,50,24Col8ksi,Level_3,58,563,164.04,3300.0,126.0,164.04,3300.0,276.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
2,55,24Col8ksi,Level_2,588,31,164.04,2940.0,0.0,164.04,2940.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
3,58,24Col8ksi,Level_3,31,561,164.04,2940.0,126.0,164.04,2940.0,276.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,59,24Col8ksi,Level_2,2116,72,164.04,2652.0,0.0,164.04,2652.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10


In [51]:
display(frames_df[(frames_df["Point1X"] == frames_df["Point2X"] ) & (frames_df["Point1Y"] == frames_df["Point2Y"] )])

,MyNames,PropName,StoryName,PointName1,PointName2,Point1X,Point1Y,Point1Z,Point2X,Point2Y,Point2Z,Angle,Offset1X,Offset2X,Offset1Y,Offset2Y,Offset1Z,Offset2Z,CardinalPoint
0,47,24Col8ksi,Level_2,587,58,164.04,3300.0,0.0,164.04,3300.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1,50,24Col8ksi,Level_3,58,563,164.04,3300.0,126.0,164.04,3300.0,276.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
2,55,24Col8ksi,Level_2,588,31,164.04,2940.0,0.0,164.04,2940.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
3,58,24Col8ksi,Level_3,31,561,164.04,2940.0,126.0,164.04,2940.0,276.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,59,24Col8ksi,Level_2,2116,72,164.04,2652.0,0.0,164.04,2652.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,2,24Col8ksi,Level_3,72,154,164.04,2652.0,126.0,164.04,2652.0,276.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1565,324,14Col8ksi,Level_2,80,157,293.04,462.0,0.0,293.04,462.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1566,325,14Col8ksi,Level_2,207,404,293.04,774.0,0.0,293.04,774.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
1592,54,24Col8ksi,Level_2,1025,468,3728.04,2580.0,0.0,3728.04,2580.0,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10


In [74]:
# filter for columns (z is allways reserved for vetical axis in  ETABs)
cols_df = frames_df[(frames_df["Point1X"] == frames_df["Point2X"] ) & (frames_df["Point1Y"] == frames_df["Point2Y"]) ]

# find lowest level columns 
# in program User will be prompted to chose the level they want to transfer
cols_df = cols_df[cols_df["StoryName"] == "Level_2"]

bot_cols = cols_df["MyNames"].unique().tolist()

def convert_to_system_string(p_list):
    return Array[String](p_list)


# print(type(convert_to_system_string(bot_cols[0])))

<class 'System.String[]'>


In [107]:
Results = cAnalysisResults(SapModel.Results)
Setup = cAnalysisResultsSetup(Results.Setup)

#select Load Cases for output
ret = Setup.DeselectAllCasesAndCombosForOutput()
ret = Setup.SetCaseSelectedForOutput('Dead')
print(ret)

0


In [111]:
#get Frame Forces for Columns
ObjectElm = 1

 #declare variables 
Name = bot_cols[0]
NumberResults = 0
Obj = []
ObjSta = []
Elm = []
ElmSta = []
LoadCase = []
StepType = []
StepNum = []
P = []
V2 = []
V3 = []
T = []
M2 = []
M3 = []




[
    ret,
    NumberResults,
    Obj,
    ObjSta,
    Elm,
    ElmSta,
    LoadCase,
    StepType,
    StepNum,
    P,
    V2,
    V3,
    T,
    M2,
    M3
]=Results.FrameForce(
    Name,
    eItemTypeElm(2),
    NumberResults,
    Obj,
    ObjSta,
    Elm,
    ElmSta,
    LoadCase,
    StepType,
    StepNum,
    P,
    V2,
    V3,
    T,
    M2,
    M3
)


print(f"analysis for: {Name}")
print(f"ret: {ret}")
    

analysis for: 47
ret: 1


In [106]:
# explore results from Flame element
print(ret)
print(NumberResults)
print(P)

print(LoadCase)

1
0
System.Double[]
System.String[]


IndexError: array index out of range